In [100]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.autograd import Variable
from torch.utils.data import DataLoader

import numpy as np

import pickle

from datasets import Multimodal_Binary_Dataset, MultimodalDataset
from fusion_model import CP_Tensor_Fusion_Network
from model import TFN

import time

In [107]:
def train_cmu_mosi(batch_size=32, epochs=100, lr=.001, max_rank=5, rank_adaptive=False,  
                   warmup_epochs=50, kl_multiplier=1e-4, no_kl_epochs=5, accelerated=True):

    # load dataset file
    file = open('../../dataset/cmu-mosi/mosi_20_seq_data.pkl', 'rb')
    data = pickle.load(file)
    file.close()

    # prepare the datasets and data loaders
    train_set = MultimodalDataset(data['train']['text'], data['train']['audio'],
                                  data['train']['vision'], data['train']['labels'])
    valid_set = MultimodalDataset(data['valid']['text'], data['valid']['audio'],
                                  data['valid']['vision'], data['valid']['labels'])

    train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_set, batch_size=len(valid_set))

    # set up model
    input_sizes = (train_set[0][0]['audio'].shape[0], train_set[0][0]['vision'].shape[0],
                   train_set[0][0]['text'].shape[1])
    hidden_sizes = (32, 32, 128)
    output_size = 1
    
    # model = CP_Tensor_Fusion_Network(input_sizes, hidden_sizes, output_size, max_rank,
    #                                  rank_adaptive)
    
    model = TFN(input_dims=input_sizes, hidden_dims=hidden_sizes, text_out=128, dropouts=(0.3, 0.3, 0.3, 0.3), 
                post_fusion_dim=1)
    
    # set up training
    DTYPE = torch.FloatTensor
    optimizer = optim.Adam(list(model.parameters()), lr=lr)
    criterion = nn.MSELoss()
    # criterion = nn.MSELoss()
    
    # train and validate
    for e in range(1, epochs + 1):
        # train
        tic = time.time()
        model.train()
        train_loss = 0.0
        for batch in train_dataloader:
            model.zero_grad()

            features, label = batch
            
            x_a = Variable(features['audio'].float().type(DTYPE), requires_grad=False)
            x_v = Variable(features['vision'].float().type(DTYPE), requires_grad=False)
            x_t = Variable(features['text'].float().type(DTYPE), requires_grad=False)
            y = Variable(label.view(-1, 1).float().type(DTYPE), requires_grad=False)
            
            output = model(x_a, x_v, x_t)
            loss = criterion(output, y)
            
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
        
        print('Train Loss {:.3f}'.format(train_loss))
        '''
        print(model.audio_subnet.linear_2.weight.grad.mean())
        print(model.video_subnet.linear_2.weight.grad.mean())
        print(model.text_subnet.linear_1.weight.grad.mean())
        print(model.tensor_fusion_layer.weight_tensor.factors[0].grad.mean())
        '''

        # validate
        model.eval()
        for batch in valid_dataloader:
            features, label = batch
            x_a = Variable(features['audio'].float().type(DTYPE), requires_grad=False)
            x_v = Variable(features['vision'].float().type(DTYPE), requires_grad=False)
            x_t = Variable(features['text'].float().type(DTYPE), requires_grad=False)
            y = Variable(label.view(-1, 1).float().type(DTYPE), requires_grad=False)

            output = model(x_a, x_v, x_t)
        
        valid_mse = nn.functional.mse_loss(output, y).item()
        print("Valid MSE {:.3f}".format(valid_mse))

In [ ]:
train_cmu_mosi()

Train Loss 92.932
Valid MSE 2.567


In [39]:
model

In [48]:
y.shape

torch.Size([32, 1])

In [1]:
import torch
import torch.nn as nn
from scipy.stats import halfcauchy, loguniform, norm, truncnorm
import numpy as np

from torch.distributions.half_cauchy import HalfCauchy
from torch.distributions.normal import Normal

In [2]:
class Adaptive_CP_Linear(nn.Module):
    
    def __init__(self, input_sizes, output_size, max_rank):
        
        super(Adaptive_CP_Linear, self).__init__()
        
        self.input_sizes = input_sizes
        self.output_size = output_size
        
        shape = input_sizes + (output_size,)
        self.weight = CP(shape, max_rank)
        
    def forard(self, inputs):
        y = 1.0
        for i, x in enumerate(inputs):
            y = y * (x @ self.weight_tensor_factors[i])
        y = y @ self.weight_tensor_factors[-1].T
        return y

In [3]:
class CP(nn.Module):
    
    def __init__(self, shape, max_rank, prior_type='log_uniform', eta=None):
        
        super(CP, self).__init__()
        
        self.shape = shape
        self.order = len(shape)
        self.max_rank = max_rank
        self.prior_type = prior_type
        
        self.factors = nn.ParameterList([nn.init.xavier_normal_(nn.Parameter(torch.empty(s, max_rank)))
                                         for s in shape])
        
        self.rank_params = nn.Parameter(torch.rand(max_rank))
        
    def log_priors(self):
        
        log_priors = 0.0
        
        rank_params_dist = HalfCauchy(1.0)
        log_priors += rank_params_dist.log_prob(self.rank_params).sum()
        
        factors_dist = Normal(0, self.rank_params)
        for f in self.factors:
            log_priors += factors_dist.log_prob(f).sum()
        
        return log_priors
    '''
    def init_factors(self):
        
        target_stddev = np.sqrt(2/np.prod(self.shape[:-1]))
        factor_stddev = np.power(target_stddev / self.max_rank, 1 / self.order)
        init_dist = truncnorm(a=-3.0*factor_stddev, b=3.0*factor_stddev, 
                              loc=0.0, scale=factor_stddev)
        for s in self.shape:
            self.factors.append(nn.Parameter(torch.tensor(init_dist.rvs((s, self.max_rank)), 
                                                          dtype=torch.float32)))
    '''

In [45]:
label.shapeb

torch.Size([32, 1, 1])

In [9]:
import torch.distributions as td

In [10]:
normal = td.Normal(0, 1)

In [22]:
normal.cdf(torch.tensor(-1))

tensor(0.1587)

In [28]:
tensor = model.tensor_fusion_layer.weight_tensor

In [29]:
factor_scale = 1e-9
factor_dist = td.Normal(loc=0, scale=tensor.rank_parameter)

In [34]:
tensor.factors[0][0]

tensor([ 0.1616,  0.2713, -0.1114,  0.0177,  0.2770,  0.1499, -0.0087,  0.0943,
         0.1177, -0.0481,  0.0643,  0.1426, -0.0518, -0.0145,  0.0151,  0.0486,
        -0.0546, -0.0239,  0.3509,  0.0476], grad_fn=<SelectBackward>)

In [38]:
factor_dist.cdf(tensor.factors[0][0])

tensor([[0.8359, 0.9501, 0.2737, 0.5453, 0.9428, 0.8083, 0.4792, 0.7029, 0.7608,
         0.3821, 0.6439, 0.7983, 0.3786, 0.4666, 0.5341, 0.6188, 0.3771, 0.4454,
         0.9798, 0.6127]], grad_fn=<MulBackward0>)

In [37]:
factor_dist.cdf(tensor.factors[0])

tensor([[0.8359, 0.9501, 0.2737, 0.5453, 0.9428, 0.8083, 0.4792, 0.7029, 0.7608,
         0.3821, 0.6439, 0.7983, 0.3786, 0.4666, 0.5341, 0.6188, 0.3771, 0.4454,
         0.9798, 0.6127],
        [0.6176, 0.7911, 0.2937, 0.6401, 0.9798, 0.4173, 0.8173, 0.7636, 0.7208,
         0.9178, 0.8128, 0.9612, 0.8145, 0.4543, 0.4445, 0.5084, 0.4040, 0.1602,
         0.2736, 0.2127],
        [0.8112, 0.4306, 0.7227, 0.6602, 0.8213, 0.0386, 0.6663, 0.5947, 0.1200,
         0.3541, 0.3249, 0.9399, 0.2074, 0.6484, 0.4268, 0.1778, 0.4798, 0.7598,
         0.0732, 0.3149],
        [0.8229, 0.0717, 0.2684, 0.0650, 0.8956, 0.8945, 0.6467, 0.2294, 0.1787,
         0.4025, 0.0838, 0.3088, 0.2935, 0.4559, 0.4754, 0.9647, 0.0382, 0.0778,
         0.4810, 0.0702],
        [0.7859, 0.1282, 0.0739, 0.6209, 0.9866, 0.7137, 0.7140, 0.5657, 0.6711,
         0.1053, 0.0805, 0.9103, 0.1144, 0.4349, 0.8268, 0.9001, 0.5141, 0.6955,
         0.5202, 0.9097],
        [0.1704, 0.8412, 0.0365, 0.0144, 0.4696, 0.7992, 0.4

In [32]:
print(tensor.rank_parameter)

Parameter containing:
tensor([[0.1653, 0.1648, 0.1852, 0.1560, 0.1755, 0.1720, 0.1666, 0.1770, 0.1661,
         0.1602, 0.1742, 0.1707, 0.1674, 0.1738, 0.1759, 0.1608, 0.1744, 0.1742,
         0.1711, 0.1663]])
